## Marketing Strategy Analysis
#### Programming Script and Technical Report

### Table of Content
1. Introduction
2. Data Loading and Quality 
3. Feature Additions and Engineering
4. Exploratory Data Analysis and Statistical Analysis
5. Final Recommendations (Optimal Sales)

### 1. Introduction

- What is the impact of each marketing strategy and sales visit on Sales (Amount Collected)?
- Is the same strategy valid for all the different client types ?


### 2. Data Loading and Quality Checks

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats

In [6]:
data=pd.read_csv('Campaign-Data.csv')
data.columns

Index(['Client ID', 'Client Type', 'Number of Customers', 'Montly Target',
       'Zip Code', 'Calendardate', 'Amount Collected', 'Unit Sold',
       'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5', 'Number of Competition'],
      dtype='object')

In [7]:
data.head()

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition
0,ID-987275,Medium Facility,2800,125,1003,16-01-2014,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
1,ID-987275,Medium Facility,2800,125,1003,16-02-2014,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low
2,ID-987275,Medium Facility,2800,125,1003,18-03-2014,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low
3,ID-987275,Medium Facility,2800,125,1003,18-04-2014,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low
4,ID-987275,Medium Facility,2800,125,1003,19-05-2014,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low


### 3. Feature Additions and Engineering

In [8]:
## Creation of Additional Features 
data['Calendardate']=pd.to_datetime(data['Calendardate'])
data['Calendar_Month']=data['Calendardate'].dt.month
data['Calendar_Year']=data['Calendardate'].dt.year

C:\Users\callu\AppData\Local\Temp\ipykernel_11584\470255435.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Calendardate']=pd.to_datetime(data['Calendardate'])


### 4. Exploratory Data Analysis and Statistical Analysis
We can have a detailed exploration that can be added to this section, but since we only need to answer three questions:

<b> 4.1 Exploring and Understanding basics data </b>

1. Distribution of Data across different accounts
2. Difference of Sales in Account Types (Using Categorical Mean)

<b> 4.2 Statistical Analysis - Answering the Questions</b>
1. Impact of Marketing Strategy on Sales (Using Correlation, Regression and Decision Tree)
2. Impact of Competition on Sales
3. How different types of client can have different strategies (Broken down Question 1 and Question 2 based on Account Type)

### 4.2 Impact of Marketing Strategy on Sales 

In [26]:
data = pd.DataFrame(data)
print(data.dtypes)

Client ID                        object
Client Type                      object
Number of Customers               int64
Montly Target                     int64
Zip Code                          int64
Calendardate             datetime64[ns]
Amount Collected                  int64
Unit Sold                         int64
Campaign (Email)                float64
Campaign (Flyer)                float64
Campaign (Phone)                float64
Sales Contact 1                 float64
Sales Contact 2                 float64
Sales Contact 3                 float64
Sales Contact 4                 float64
Sales Contact 5                 float64
Number of Competition           float64
Calendar_Month                    int32
Calendar_Year                     int32
dtype: object


#### Understanding of distrubtions

In [9]:
data['Client Type'].value_counts(normalize=True)

Client Type
Large Facility      0.459677
Small Facility      0.282258
Medium Facility     0.169355
Private Facility    0.088710
Name: proportion, dtype: float64

In [10]:
pd.crosstab(data['Number of Competition'], data['Client Type'], margins=True, normalize='columns')

Client Type,Large Facility,Medium Facility,Private Facility,Small Facility,All
Number of Competition,,,,,
High,0.166667,0.166667,0.166667,0.166667,0.166667
Low,0.833333,0.833333,0.833333,0.833333,0.833333


In [27]:
# Select only numeric columns for aggregation, excluding 'Number of Competition'
numeric_cols = data.select_dtypes(include='number').columns.tolist()
numeric_cols.remove('Number of Competition')

In [32]:
data.groupby('Number of Competition').mean()

In [16]:
data.groupby('Client Type').mean()

TypeError: agg function failed [how->mean,dtype->object]

In [37]:
data.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['Client ID', 'Client Type', 'Number of Customers', 'Montly Target',
       'Zip Code', 'Calendardate', 'Amount Collected', 'Unit Sold',
       'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5', 'Number of Competition',
       'Calendar_Month', 'Calendar_Year'],
      dtype='object')>

In [38]:
data.drop('Client ID').corr()[['Amount_Collected']]

KeyError: "['Client ID'] not found in axis"

Correlation Analysis

In [41]:
## Consolidated Strategy for Targeting
        
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
correlation_analysis=pd.DataFrame(data[['Amount Collected',
'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected']).reset_index()
correlation_analysis.columns=['Impacting Variable','Degree of Linear Impact (Correlation)']
correlation_analysis=correlation_analysis[correlation_analysis['Impacting Variable']!='Amount Collected']
correlation_analysis=correlation_analysis.sort_values('Degree of Linear Impact (Correlation)',ascending=False)
correlation_analysis.style.background_gradient(cmap=cm).set_precision(2)

AttributeError: 'Styler' object has no attribute 'set_precision'

#### Market Strategy Impact on Sales (Broken by different account type)

In [ ]:
# Import seaborn library
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
correlation_analysis=pd.DataFrame(data.groupby('Client Type')[['Amount Collected',
       'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)',
       'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3',
       'Sales Contact 4', 'Sales Contact 5']].corr()['Amount Collected']).reset_index()
correlation_analysis=correlation_analysis.sort_values(['Client Type','Amount Collected'],ascending=False)
correlation_analysis.columns=['Acc Type','Variable Impact on Sales','Impact']
correlation_analysis=correlation_analysis[correlation_analysis['Variable Impact on Sales']!='Amount Collected'].reset_index(drop=True)
correlation_analysis.style.background_gradient(cmap=cm).set_precision(2)

#### Regression Analysis (Market Sales and Strategies)

In [43]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
data.columns=[mystring.replace(" ", "_") for mystring in data.columns]
data.columns=[mystring.replace("(", "") for mystring in data.columns]
data.columns=[mystring.replace(")", "") for mystring in data.columns]
results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5',data=data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     342.1
Date:                Wed, 22 May 2024   Prob (F-statistic):               0.00
Time:                        09:57:32   Log-Likelihood:                -54512.
No. Observations:                2976   AIC:                         1.090e+05
Df Residuals:                    2967   BIC:                         1.091e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.481e+06   5.12e+05     

In [45]:
df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0]

C:\Users\callu\AppData\Local\Temp\ipykernel_11584\436344858.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0]


In [46]:
df=df.reset_index()
df=df[df['P>|t|']<0.05][['index','coef']]
df

,index,coef
0,Intercept,1.481000e+06
2,Campaign_Flyer,3.337600e+00
4,Sales_Contact_1,4.236800e+00
5,Sales_Contact_2,3.638200e+00
6,Sales_Contact_3,2.343200e+00
7,Sales_Contact_4,1.094780e+01


#### Regression Analysis (Market Sales and Strategies) - Broken for different account types

In [52]:
consolidated_summary=pd.DataFrame()
for acctype in list(set(list(data['Client_Type']))):
    temp_data=data[data['Client_Type']==acctype].copy()
    results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5', data=temp_data).fit()
    df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
    df=df[df['P>|t|']<0.05][['index','coef']]
    df.columns=['Variable','Coefficent (Impact)']
    df['Account Type']=acctype
    df=df.sort_values('Coefficent (Impact)',ascending=False)
    df=df[df['Variable']!='Intercept']
    print(acctype)
   #  consolidated_summary=consolidated_summary.concat(df)
    consolidated_summary = pd.concat([consolidated_summary, df])
    print(df)
    #print(results.summary())
    

C:\Users\callu\AppData\Local\Temp\ipykernel_11584\2475560057.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()


Medium Facility
          Variable  Coefficent (Impact)     Account Type
2   Campaign_Flyer               4.1059  Medium Facility
5  Sales_Contact_2               3.5778  Medium Facility
4  Sales_Contact_1               3.1365  Medium Facility
6  Sales_Contact_3               2.1174  Medium Facility
Small Facility 
          Variable  Coefficent (Impact)     Account Type
5  Sales_Contact_2             0.810100  Small Facility 
3   Campaign_Phone            -0.000003  Small Facility 


C:\Users\callu\AppData\Local\Temp\ipykernel_11584\2475560057.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()


Private Facility
          Variable  Coefficent (Impact)      Account Type
5  Sales_Contact_2               6.6223  Private Facility
Large Facility 
          Variable  Coefficent (Impact)     Account Type
4  Sales_Contact_1              11.6731  Large Facility 
7  Sales_Contact_4              10.6145  Large Facility 
5  Sales_Contact_2               4.0031  Large Facility 
2   Campaign_Flyer               2.7204  Large Facility 
6  Sales_Contact_3               2.0316  Large Facility 
3   Campaign_Phone              -3.5361  Large Facility 


C:\Users\callu\AppData\Local\Temp\ipykernel_11584\2475560057.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
C:\Users\callu\AppData\Local\Temp\ipykernel_11584\2475560057.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()


In [53]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
consolidated_summary=pd.DataFrame()
for acctype in list(set(list(data['Client_Type']))):
    print(acctype)
    temp_data=data[data['Client_Type']==acctype].copy()
    results = smf.ols('Amount_Collected ~ Campaign_Email+Campaign_Flyer+Campaign_Phone+\
       Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3+Sales_Contact_4 + Sales_Contact_5', data=temp_data).fit()
    df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
    df=df[df['P>|t|']<0.05][['index','coef']]
    df.columns=['Variable','Coefficent (Impact)']
    df['Account Type']=acctype
    df=df.sort_values('Coefficent (Impact)',ascending=False)
    df=df[df['Variable']!='Intercept']
    consolidated_summary = pd.concat([consolidated_summary, df])
    print(results.summary())

Medium Facility


C:\Users\callu\AppData\Local\Temp\ipykernel_11584\661246422.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()


                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     48.00
Date:                Wed, 22 May 2024   Prob (F-statistic):           4.13e-57
Time:                        10:02:20   Log-Likelihood:                -9461.9
No. Observations:                 504   AIC:                         1.894e+04
Df Residuals:                     495   BIC:                         1.898e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        5.682e+06   2.53e+06     

C:\Users\callu\AppData\Local\Temp\ipykernel_11584\661246422.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()
C:\Users\callu\AppData\Local\Temp\ipykernel_11584\661246422.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()


                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     6.829
Date:                Wed, 22 May 2024   Prob (F-statistic):           6.63e-08
Time:                        10:02:20   Log-Likelihood:                -14125.
No. Observations:                 840   AIC:                         2.827e+04
Df Residuals:                     832   BIC:                         2.830e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        8.789e+05   2.07e+05     

C:\Users\callu\AppData\Local\Temp\ipykernel_11584\661246422.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0].reset_index()


### 5. Final Recommendations

Using the below table we can use the coefficent to see how much return we can derive from each dollar we spend, here we can clearly see that for different account type different Campaigns and Different Sales Contact are effective with different extend. 

<b>Case Explanation - Medium Facility </b><br>
For Example Medium Facility shows decent results with Flyer Campiagns and each dollar spend return 4 dollars on average. Sales Contact 2 is highly effective followed by Sales Contact 1 and Sales Contact 3. Else all other strategy shows no impact can be dropped to save cost. 


In [54]:
consolidated_summary

,Variable,Coefficent (Impact),Account Type
2,Campaign_Flyer,4.105900,Medium Facility
5,Sales_Contact_2,3.577800,Medium Facility
4,Sales_Contact_1,3.136500,Medium Facility
6,Sales_Contact_3,2.117400,Medium Facility
5,Sales_Contact_2,0.810100,Small Facility
3,Campaign_Phone,-0.000003,Small Facility
5,Sales_Contact_2,6.622300,Private Facility
4,Sales_Contact_1,11.673100,Large Facility
7,Sales_Contact_4,10.614500,Large Facility
5,Sales_Contact_2,4.003100,Large Facility


In [55]:
consolidated_summary.reset_index(inplace=True)
consolidated_summary.drop('index',inplace=True,axis=1)

In [56]:
consolidated_summary.columns = ['Variable','Return on Investment','Account Type']
consolidated_summary['Return on Investment']= consolidated_summary['Return on Investment'].apply(lambda x: round(x,1))
consolidated_summary.style.background_gradient(cmap='RdYlGn')

,Variable,Return on Investment,Account Type
0,Campaign_Flyer,4.100000,Medium Facility
1,Sales_Contact_2,3.600000,Medium Facility
2,Sales_Contact_1,3.100000,Medium Facility
3,Sales_Contact_3,2.100000,Medium Facility
4,Sales_Contact_2,0.800000,Small Facility
5,Campaign_Phone,-0.000000,Small Facility
6,Sales_Contact_2,6.600000,Private Facility
7,Sales_Contact_1,11.700000,Large Facility
8,Sales_Contact_4,10.600000,Large Facility
9,Sales_Contact_2,4.000000,Large Facility


In [57]:
import seaborn as sns
import matplotlib.pyplot as plt 

In [58]:
def format(x):
        return "${:.1f}".format(x)
consolidated_summary['Return on Investment']  = consolidated_summary['Return on Investment'].apply(format)

In [59]:
consolidated_summary.columns = ['Variable','Return on Investment','Account Type']
consolidated_summary.style.background_gradient(cmap='RdYlGn')

,Variable,Return on Investment,Account Type
0,Campaign_Flyer,$4.1,Medium Facility
1,Sales_Contact_2,$3.6,Medium Facility
2,Sales_Contact_1,$3.1,Medium Facility
3,Sales_Contact_3,$2.1,Medium Facility
4,Sales_Contact_2,$0.8,Small Facility
5,Campaign_Phone,$-0.0,Small Facility
6,Sales_Contact_2,$6.6,Private Facility
7,Sales_Contact_1,$11.7,Large Facility
8,Sales_Contact_4,$10.6,Large Facility
9,Sales_Contact_2,$4.0,Large Facility


In [ ]:
consolidated_summary.to_csv('consolidated_summary.csv')
files.download('consolidated_summary.csv')